# **Building a Conversational Agent with Context Awareness**
##  Overview
This Code outlines the process of creating a conversational agent that maintains context across multiple interactions. We'll use a modern AI framework to build an agent capable of engaging in more natural and coherent conversations.
## Motivation
Many simple chatbots lack the ability to maintain context, leading to disjointed and frustrating user experiences. This tutorial aims to solve that problem by implementing a conversational agent that can remember and refer to previous parts of the conversation, enhancing the overall interaction quality.

## **Key Components**  
 1. **Language Model:** The core AI component that generates responses.
 2. **Prompt Template:** Defines the structure of our conversations.
 3. **History Manager:** Manages conversation history and context.
 4. **Message Store:**  Stores the messages for each conversation session.

# **Method Details**
## Setting Up the Environment
  Begin by setting up the necessary AI framework and ensuring access to a suitable language model. This forms the foundation of our conversational agent.
## Defining the Conversation Structure
  Create a template that includes:

 - A system message defining the AI's role
 - A placeholder for conversation history
 - The user's input

 This structure guides the AI's responses and maintains consistency throughout the conversation.

## **Building the Conversational Chain**
Combine the prompt template with the language model to create a basic conversational chain. Wrap this chain with a history management component that automatically handles the insertion and retrieval of conversation history.  

## Interacting with the Agent
To use the agent, invoke it with a user input and a session identifier. The history manager takes care of retrieving the appropriate conversation history, inserting it into the prompt, and storing new messages after each interaction.
# Conclusion
This approach to creating a conversational agent offers several advantages:

- **Context Awareness:** The agent can refer to previous parts of the conversation, leading to more natural interactions.
- **Simplicity:** The modular design keeps the implementation straightforward.
- **Flexibility:** It's easy to modify the conversation structure or switch to a different language model.
- **Scalability:** The session-based approach allows for managing multiple independent conversations.

With this foundation, you can further enhance the agent by:

- Implementing more sophisticated prompt engineering
- Integrating it with external knowledge bases
- Adding specialized capabilities for specific domains
- Incorporating error handling and conversation repair strategies

By focusing on context management, this conversational agent design significantly improves upon basic chatbot functionality, paving the way for more engaging and helpful AI assistants.

## Import required libraries


In [ ]:
!pip install langchain langchain_community langchain_core

In [3]:
from langchain.llms import  HuggingFaceHub
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from dotenv import load_dotenv
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('hugginface_key')

## **Load environment variables and initialize the language model**

In [4]:
load_dotenv()
llm  =  HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-beta",
                       model_kwargs={"temperature":0.5, "max_length":512})


<ipython-input-4-c4462cf3fbeb>:2: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm  =  HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-beta",


### Create a simple in-memory store for chat histories


In [5]:
store = {}
def get_chat_history(session_id: str) :
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


### Create the prompt template


In [31]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [32]:
prompt

ChatPromptTemplate(input_variables=['history', 'input'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_c

# **Combine the prompt and model into a runnable chain**


In [8]:
chain = prompt | llm

In [9]:
chain

ChatPromptTemplate(input_variables=['history', 'input'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_c

## Wrap the chain with message history


In [10]:

chain_with_history = RunnableWithMessageHistory(
    chain ,
    get_chat_history,
    input_messages_key="input",
    output_messages_key="history"
)

## Example usage


In [ ]:
session_id = "user_123"


response1 = chain_with_history.invoke(
    {"input": "Hello! How are you?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1["Human"])

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2["Human"])

## **Print the Convertion History :**

In [34]:

print("\nConversation History:")
for message in store[session_id].messages:
    print(f"{message.type}: {message.content}")


Conversation History:
